<a href="https://colab.research.google.com/github/itc202/praktik/blob/main/ProtoNet_%D0%BD%D0%B0_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

оригинальная модель Protonet предложена в статье Prototypical Networks for Few-shot Learning появилась на arhiv.org в 2017
Одно из улучшений в статье 2023 https://www.sciencedirect.com/science/article/abs/pii/S0167865523001940 (имеется pdf версия) , в статье Plant disease recognition in a low data scenario using few-shot learning от 2024 https://www.sciencedirect.com/science/article/pii/S0168169924002035  и других статьях.


**Прототипические сети Protonet используются для задач классификации, особенно в контексте обучения с малым количеством примеров.** Они работают, вычисляя прототипы классов и классифицируя новые примеры на основе расстояния до этих прототипов. Вот пример реализации прототипической сети на Keras:

В этом примере мы будем использовать набор данных CIFAR-10, который можно загрузить непосредственно из Keras. Мы будем обучать прототипическую сеть, используя изображения из этого набора данных.
python




Как это работает:
Метод call:
Когда вы передаете изображения в модель, они сначала обрабатываются с помощью слоя Flatten, который преобразует 3D-изображения (например, 32x32x3 для CIFAR-10) в 1D-векторы.

Затем эти векторы передаются в плотный слой (Dense), который применяет линейное преобразование с активацией ReLU. Этот слой фактически и **извлекает признаки изображений**, создавая эмбеддинги, которые представляют собой высокоуровневые характеристики изображений.

Эмбеддинги:
Эмбеддинги, полученные из метода call, используются для дальнейших вычислений, таких как определение прототипов классов и расстояний между эмбеддингами и прототипами.

Таким образом, именно в методе call происходит извлечение признаков изображений CIFAR-10 с помощью плотного слоя, который обучается в процессе тренировки модели.

В варианте кода ниже нет сверточных слоев. Более сллоожный вариант с кодировщиком на свреточных слоях рпиведен в конце ноутбука
Объяснение:
Архитектура: **В оригинальной реализации прототипических сетей**, как правило, используется свёрточная нейронная сеть (CNN) в качестве кодировщика для извлечения высокоуровневых признаков из изображений. Это позволяет модели эффективно обрабатывать визуальные данные, учитывая пространственные зависимости и иерархию признаков.

Преимущества свёрточных слоёв: Свёрточные слои хорошо подходят для задач компьютерного зрения, так как они могут выявлять локальные паттерны и структуры в изображениях, что значительно улучшает качество извлечения признаков по сравнению с простыми плотными слоями.

Таким образом, в отличие от упрощенной версии прототипической сети, представленной в этом  коде, оригинальная модель использует свёрточные слои для более эффективного извлечения признаков из изображений.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import numpy as np

In [ ]:
class PrototypicalNetwork(tf.keras.Model):
    def __init__(self, embedding_dim, num_classes):
        super(PrototypicalNetwork, self).__init__()
        self.embedding = layers.Flatten()
        self.dense = layers.Dense(embedding_dim, activation='relu')
        self.num_classes = num_classes

    def call(self, inputs):
        # Получаем эмбеддинги
        embeddings = self.dense(self.embedding(inputs))
        return embeddings

    def compute_prototypes(self, embeddings, labels):
        # Вычисляем прототипы для каждого класса
        prototypes = []
        for i in range(self.num_classes):
            class_embeddings = embeddings[labels == i]
            if len(class_embeddings) > 0:
                prototype = tf.reduce_mean(class_embeddings, axis=0)
                prototypes.append(prototype)
        return tf.stack(prototypes)

    def compute_distances(self, embeddings, prototypes):
        # Вычисляем расстояния между эмбеддингами и прототипами
        distances = tf.norm(tf.expand_dims(embeddings, 1) - prototypes, axis=2)
        return distances

In [ ]:
def train_prototypical_network(train_data, train_labels, embedding_dim=64, num_classes=10, epochs=10, batch_size=32):
    model = PrototypicalNetwork(embedding_dim, num_classes)
    optimizer = tf.keras.optimizers.Adam()

    for epoch in range(epochs):
        for i in range(0, len(train_data), batch_size):
            batch_data = train_data[i:i + batch_size]
            batch_labels = train_labels[i:i + batch_size]

            with tf.GradientTape() as tape:
                # Получаем эмбеддинги
                embeddings = model(batch_data)
                # Вычисляем прототипы
                prototypes = model.compute_prototypes(embeddings, batch_labels)
                # Вычисляем расстояния
                distances = model.compute_distances(embeddings, prototypes)
                # Получаем потери
                loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=-distances, labels=batch_labels))

            # Обновляем веса
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.numpy()}')

In [ ]:
# Загружаем данные CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0  # Нормализация
x_test = x_test.astype('float32') / 255.0  # Нормализация
y_train = y_train.flatten()  # Преобразуем в одномерный массив
y_test = y_test.flatten()  # Преобразуем в одномерный массив

In [ ]:
# Обучение модели
train_prototypical_network(x_train, y_train, epochs=10)

Объяснение изменений
Загрузка CIFAR-10: Мы используем cifar10.load_data() для загрузки набора данных CIFAR-10 напрямую из Keras. Данные нормализуются для лучшей работы модели.
Модель PrototypicalNetwork: Мы изменили архитектуру модели, чтобы она использовала Flatten для обработки изображений размером 32x32x3 из CIFAR-10. Затем добавляется плотный слой для получения эмбеддингов.
Обучение: Обучение происходит поэтапно с использованием батчей, что позволяет эффективно обрабатывать данные.
Этот код создает и обучает прототипическую сеть на наборе данных CIFAR-10, используя изображения и метки, доступные в Keras. Вы можете адаптировать параметры, такие как размер эмбеддинга, количество эпох и размер батча, в зависимости от ваших потребностей.

In [ ]:
# Функция для загрузки и предобработки пользовательских изображений
def load_and_preprocess_image(image_path, target_size=(32, 32)):
    img = Image.open(image_path)
    img = img.resize(target_size)  # Изменяем размер
    img_array = np.array(img) / 255.0  # Нормализация
    img_array = np.expand_dims(img_array, axis=0)  # Добавляем размерность для батча
    return img_array

In [ ]:
# Функция для классификации пользовательских изображений
def classify_custom_images(model, image_paths, num_classes):
    embeddings = []
    for img_path in image_paths:
        img_array = load_and_preprocess_image(img_path)
        embedding = model(img_array)  # Получаем эмбеддинг
        embeddings.append(embedding)

    embeddings = tf.concat(embeddings, axis=0)
    prototypes = model.compute_prototypes(embeddings, np.zeros(len(embeddings)))  # Используем нулевые метки для прототипов
    distances = model.compute_distances(embeddings, prototypes)

    # Получаем предсказанные классы
    predictions = tf.argmin(distances, axis=1)
    return predictions.numpy()


In [ ]:
# Пример использования для классификации пользовательских изображений
custom_image_paths = ['path_to_your_image1.jpg', 'path_to_your_image2.jpg']  # Замените на свои пути к изображениям
predicted_classes = classify_custom_images(model, custom_image_paths, num_classes=10)

for img_path, pred_class in zip(custom_image_paths, predicted_classes):
    print(f'Image: {img_path}, Predicted Class: {pred_class}')

Ниже весь код, предложенный GPT на заапрос с класссифкацией своих изображений

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import numpy as np
from tensorflow.keras.preprocessing import image
import os
from PIL import Image

class PrototypicalNetwork(tf.keras.Model):
    def __init__(self, embedding_dim, num_classes):
        super(PrototypicalNetwork, self).__init__()
        self.embedding = layers.Flatten()
        self.dense = layers.Dense(embedding_dim, activation='relu')
        self.num_classes = num_classes

    def call(self, inputs):
        # Получаем эмбеддинги
        embeddings = self.dense(self.embedding(inputs))
        return embeddings

    def compute_prototypes(self, embeddings, labels):
        # Вычисляем прототипы для каждого класса
        prototypes = []
        for i in range(self.num_classes):
            class_embeddings = embeddings[labels == i]
            if len(class_embeddings) > 0:
                prototype = tf.reduce_mean(class_embeddings, axis=0)
                prototypes.append(prototype)
        return tf.stack(prototypes)

    def compute_distances(self, embeddings, prototypes):
        # Вычисляем расстояния между эмбеддингами и прототипами
        distances = tf.norm(tf.expand_dims(embeddings, 1) - prototypes, axis=2)
        return distances

def train_prototypical_network(train_data, train_labels, embedding_dim=64, num_classes=10, epochs=10, batch_size=32):
    model = PrototypicalNetwork(embedding_dim, num_classes)
    optimizer = tf.keras.optimizers.Adam()

    for epoch in range(epochs):
        for i in range(0, len(train_data), batch_size):
            batch_data = train_data[i:i + batch_size]
            batch_labels = train_labels[i:i + batch_size]

            with tf.GradientTape() as tape:
                # Получаем эмбеддинги
                embeddings = model(batch_data)
                # Вычисляем прототипы
                prototypes = model.compute_prototypes(embeddings, batch_labels)
                # Вычисляем расстояния
                distances = model.compute_distances(embeddings, prototypes)
                # Получаем потери
                loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=-distances, labels=batch_labels))

            # Обновляем веса
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.numpy()}')

# Загружаем данные CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0  # Нормализация
x_test = x_test.astype('float32') / 255.0  # Нормализация
y_train = y_train.flatten()  # Преобразуем в одномерный массив
y_test = y_test.flatten()  # Преобразуем в одномерный массив

# Обучение модели
train_prototypical_network(x_train, y_train, epochs=10)

# Функция для загрузки и предобработки пользовательских изображений
def load_and_preprocess_image(image_path, target_size=(32, 32)):
    img = Image.open(image_path)
    img = img.resize(target_size)  # Изменяем размер
    img_array = np.array(img) / 255.0  # Нормализация
    img_array = np.expand_dims(img_array, axis=0)  # Добавляем размерность для батча
    return img_array

# Функция для классификации пользовательских изображений
def classify_custom_images(model, image_paths, num_classes):
    embeddings = []
    for img_path in image_paths:
        img_array = load_and_preprocess_image(img_path)
        embedding = model(img_array)  # Получаем эмбеддинг
        embeddings.append(embedding)

    embeddings = tf.concat(embeddings, axis=0)
    prototypes = model.compute_prototypes(embeddings, np.zeros(len(embeddings)))  # Используем нулевые метки для прототипов
    distances = model.compute_distances(embeddings, prototypes)

    # Получаем предсказанные классы
    predictions = tf.argmin(distances, axis=1)
    return predictions.numpy()

# Пример использования для классификации пользовательских изображений
custom_image_paths = ['path_to_your_image1.jpg', 'path_to_your_image2.jpg']  # Замените на свои пути к изображениям
predicted_classes = classify_custom_images(model, custom_image_paths, num_classes=10)

for img_path, pred_class in zip(custom_image_paths, predicted_classes):
    print(f'Image: {img_path}, Predicted Class: {pred_class}')

Объяснение добавленных частей
Функция load_and_preprocess_image: Эта функция загружает изображение, изменяет его размер до 32x32 пикселей (размер изображений в CIFAR-10) и нормализует его.

Функция classify_custom_images: Эта функция принимает список путей к пользовательским изображениям, обрабатывает их, получает эмбеддинги и вычисляет расстояния до прототипов. Затем она возвращает предсказанные классы.

Пример использования: В конце кода вы можете указать свои пути к изображениям, которые вы хотите классифицировать. Модель будет возвращать предсказанные классы для каждого изображения.
Этот код позволяет вам использовать обученную прототипическую сеть для классификации новых изображений, загружая их из ваших собственных файлов.

Обновленный код прототипической сети (Prototypical Network) с использованием свёрточных слоёв для извлечения признаков
В этом обновлённом коде мы добавим свёрточный кодировщик для извлечения признаков из изображений CIFAR-10. Это позволит модели более эффективно обрабатывать визуальные данные.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image

In [ ]:
# Определяем свёрточный кодировщик
class ConvEncoder(tf.keras.Model):
    def __init__(self):
        super(ConvEncoder, self).__init__()
        self.conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')
        self.pool1 = layers.MaxPooling2D((2, 2))
        self.conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')
        self.pool2 = layers.MaxPooling2D((2, 2))
        self.conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')
        self.pool3 = layers.MaxPooling2D((2, 2))
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(64, activation='relu')  # Эмбеддинг размерности 64

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.flatten(x)
        return self.dense(x)

In [ ]:
class PrototypicalNetwork(tf.keras.Model):
    def __init__(self, encoder, num_classes):
        super(PrototypicalNetwork, self).__init__()
        self.encoder = encoder
        self.num_classes = num_classes

    def call(self, inputs):
        return self.encoder(inputs)

    def compute_prototypes(self, embeddings, labels):
        prototypes = []
        for i in range(self.num_classes):
            class_embeddings = embeddings[labels == i]
            if len(class_embeddings) > 0:
                prototype = tf.reduce_mean(class_embeddings, axis=0)
                prototypes.append(prototype)
        return tf.stack(prototypes)

    def compute_distances(self, embeddings, prototypes):
        distances = tf.norm(tf.expand_dims(embeddings, 1) - prototypes, axis=2)
        return distances

In [ ]:
def train_prototypical_network(train_data, train_labels, num_classes=10, epochs=10, batch_size=32):
    encoder = ConvEncoder()
    model = PrototypicalNetwork(encoder, num_classes)
    optimizer = tf.keras.optimizers.Adam()

    for epoch in range(epochs):
        for i in range(0, len(train_data), batch_size):
            batch_data = train_data[i:i + batch_size]
            batch_labels = train_labels[i:i + batch_size]

            with tf.GradientTape() as tape:
                embeddings = model(batch_data)
                prototypes = model.compute_prototypes(embeddings, batch_labels)
                distances = model.compute_distances(embeddings, prototypes)
                loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=-distances, labels=batch_labels))

            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.numpy()}')

In [ ]:
# Загружаем данные CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0  # Нормализация
x_test = x_test.astype('float32') / 255.0  # Нормализация
y_train = y_train.flatten()  # Преобразуем в одномерный массив
y_test = y_test.flatten()  # Преобразуем в одномерный массив

# Обучение модели
train_prototypical_network(x_train, y_train, epochs=10)

# Функция для загрузки и предобработки пользовательских изображений
def load_and_preprocess_image(image_path, target_size=(32, 32)):
    img = Image.open(image_path)
    img = img.resize(target_size)  # Изменяем размер
    img_array = np.array(img) / 255.0  # Нормализация
    img_array = np.expand_dims(img_array, axis=0)  # Добавляем размерность для батча
    return img_array

In [ ]:
# Функция для классификации пользовательских изображений
def classify_custom_images(model, image_paths, num_classes):
    encoder = model.encoder  # Получаем кодировщик
    embeddings = []
    for img_path in image_paths:
        img_array = load_and_preprocess_image(img_path)
        embedding = encoder(img_array)  # Получаем эмбеддинг
        embeddings.append(embedding)

    embeddings = tf.concat(embeddings, axis=0)
    prototypes = model.compute_prototypes(embeddings, np.zeros(len(embeddings)))  # Используем нулевые метки для прототипов
    distances = model.compute_distances(embeddings, prototypes)

    # Получаем предсказанные классы
    predictions = tf.argmin(distances, axis=1)
    return predictions.numpy()

In [ ]:
# Пример использования для классификации пользовательских изображений
custom_image_paths = ['path_to_your_image1.jpg', 'path_to_your_image2.jpg']  # Замените на свои пути к изображениям
predicted_classes = classify_custom_images(model, custom_image_paths, num_classes=10)

for img_path, pred_class in zip(custom_image_paths, predicted_classes):
    print(f'Image: {img_path}, Predicted Class: {pred_class}')

Объяснение изменений:
Кодировщик ConvEncoder:
Добавлен новый класс ConvEncoder, который состоит из свёрточных слоёв и слоёв подвыборки (MaxPooling). Это позволяет модели извлекать более сложные признаки из изображений.

В конце кодировщика используется плотный слой для получения окончательных эмбеддингов размерности 64.

Изменение в классе PrototypicalNetwork:
Теперь класс PrototypicalNetwork принимает экземпляр ConvEncoder в качестве параметра, что позволяет использовать свёрточные слои для извлечения признаков.

Обучение и предсказание:

В процессе обучения и предсказания мы используем кодировщик для получения эмбеддингов изображений, как и раньше, но теперь с учетом свёрточных слоёв.

Функция предсказания:

Функция classify_custom_images использует кодировщик для извлечения эмбеддингов из пользовательских изображений перед вычислением прототипов и расстояний.
Этот код теперь включает свёрточные слои для извлечения признаков, что должно улучшить качество работы модели на задачах классификации.

Почему для обучения модели здесь используется своя функция train_prototypical_network, а не станадртный метод fit для класса Model . Можно ли сделать так, чтобы использовать не свою функцию обучения, а этот метод fit?

В данном коде используется собственная функция train_prototypical_network для обучения модели по нескольким причинам:

Специфика задачи: Прототипические сети требуют специфического подхода к обучению, который включает в себя вычисление прототипов для каждого класса и расстояний между эмбеддингами и этими прототипами. Это не является стандартной процедурой, поддерживаемой методом fit().

Контроль над процессом обучения: Используя собственную функцию, разработчик имеет полный контроль над процессом обучения, включая управление градиентами, вычисление потерь и обновление весов. Это позволяет более гибко настраивать процесс обучения в соответствии с требованиями модели.

Кастомизация: В данной реализации необходимо выполнять специфические операции, такие как вычисление прототипов и расстояний, которые не могут быть легко реализованы с помощью стандартного метода fit().

**Можно ли использовать метод fit()?**
Да, можно адаптировать код так, чтобы использовать метод fit(). Для этого необходимо переопределить метод **train_step** в классе модели. Это позволит интегрировать специфические операции, такие как вычисление прототипов и расстояний, в стандартный процесс обучения. Вот как это можно сделать:

In [ ]:
class PrototypicalNetwork(tf.keras.Model):
    def __init__(self, encoder, num_classes):
        super(PrototypicalNetwork, self).__init__()
        self.encoder = encoder
        self.num_classes = num_classes

    def call(self, inputs):
        return self.encoder(inputs)

    def compute_prototypes(self, embeddings, labels):
        prototypes = []
        for i in range(self.num_classes):
            class_embeddings = embeddings[labels == i]
            if len(class_embeddings) > 0:
                prototype = tf.reduce_mean(class_embeddings, axis=0)
                prototypes.append(prototype)
        return tf.stack(prototypes)

    def compute_distances(self, embeddings, prototypes):
        distances = tf.norm(tf.expand_dims(embeddings, 1) - prototypes, axis=2)
        return distances

    def train_step(self, data):
        # Разделяем данные на входы и метки
        x, y = data

        with tf.GradientTape() as tape:
            embeddings = self(x, training=True)
            prototypes = self.compute_prototypes(embeddings, y)
            distances = self.compute_distances(embeddings, prototypes)
            loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=-distances, labels=y))

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return {"loss": loss}

# Обучение модели с использованием метода fit
model = PrototypicalNetwork(ConvEncoder(), num_classes=10)
model.compile(optimizer=tf.keras.optimizers.Adam())

# Теперь можно использовать метод fit
model.fit(x_train, y_train, epochs=10, batch_size=32)

Ниже показан пример того, как ранеее оубченная модель Protonet используется дл распознавания   нового класса. Ранее оубченная модель была сохранена в формате h5. Для этого для нового класса также формируется набор примеров, по которым будет вычисляться прототип. Далее этот прототип вклоючается в список уже известных модели прототипов (предполагаем, что они известны из процесса обучения). Видно, что прототип формируется оубченной моделью с помощью perdict

ЗДЕСЬ для иллюстрации исопльзуется новый класс "человек", а моель обучалась на классах "кошка", "собака", "олень"

In [ ]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

# Загрузка обученной модели
model = load_model('path_to_your_protonet_model.h5')

# Функция для предобработки изображения
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Измените размер в зависимости от вашей модели
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Нормализация
    return img_array

# Функция для получения прототипа класса
def get_class_prototype(images):
    features = []
    for img in images:
        feature = model.predict(preprocess_image(img))
        features.append(feature)
    return np.mean(features, axis=0)

In [ ]:
# Примеры изображений для нового класса "человек"
human_examples = ['path_to_human_image1.jpg', 'path_to_human_image2.jpg']
human_prototype = get_class_prototype(human_examples)
"""
# Прототипы существующих классов
dog_prototype = ...  # Получите прототип для собак
cat_prototype = ...  # Получите прототип для кошек
deer_prototype = ...  # Получите прототип для оленей
""""
# Примеры изображений для ранее известных классов
dog_examples = [preprocess_image('path_to_dog_image1.jpg'), preprocess_image('path_to_dog_image2.jpg')]
cat_examples = [preprocess_image('path_to_cat_image1.jpg'), preprocess_image('path_to_cat_image2.jpg')]
deer_examples = [preprocess_image('path_to_deer_image1.jpg'), preprocess_image('path_to_deer_image2.jpg')]

# Получение прототипов классов
dog_prototype = get_class_prototypes(dog_examples)
cat_prototype = get_class_prototypes(cat_examples)
deer_prototype = get_class_prototypes(deer_examples)

# Обновление массива прототипов
class_prototypes = np.array([dog_prototype, cat_prototype, deer_prototype, human_prototype])

In [ ]:
# Функция для предсказания класса
def predict_class(img_path):
    img = preprocess_image(img_path)
    img_features = model.predict(img)  # Получаем вектор признаков для нового изображения
    distances = np.linalg.norm(class_prototypes - img_features, axis=1)  # Вычисляем расстояния до прототипов
    predicted_class = np.argmin(distances)  # Находим класс с минимальным расстоянием
    return predicted_class

In [ ]:
# Пример использования
test_image_path = 'path_to_test_image.jpg'
predicted_class = predict_class(test_image_path)

# Интерпретация результата
class_names = ['Dog', 'Cat', 'Deer']
print(f'The predicted class for the image is: {class_names[predicted_class]}')